In [0]:
from __future__ import division
from __future__ import print_function
import numpy as np
import scipy.sparse as sp
import torch
import math

import torch

from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

import torch.nn as nn
import torch.nn.functional as F

import time
import argparse
import numpy as np

import torch
import torch.nn.functional as F
import torch.optim as optim

import pandas as pd

from sklearn.metrics import f1_score

In [2]:
from google.colab import drive

drive.mount('/content/gdrive' ,force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


defining a few helper function....like normalissation which is needed in GCN

In [0]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx


def accuracy(output, labels):
    preds = output.max(1)[1].type_as(labels)
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return (correct / len(labels))


def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

In [0]:
def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

In [10]:
train_lim=0
val_lim=0
test_lim=0



def load_my_data():

    global train_lim,test_lim,val_lim,true_test_edges,false_test_edges,true_train_edges
    true_train_edges=pd.read_csv("/content/gdrive/My Drive/datasets/lab_as2/train_edges.csv",header=None)
    true_test_edges=pd.read_csv("/content/gdrive/My Drive/datasets/lab_as2/test_true_edges.csv")
    false_test_edges=pd.read_csv("/content/gdrive/My Drive/datasets/lab_as2/false_data_test.csv")
    label_data=pd.read_csv("/content/gdrive/My Drive/datasets/lab_as2/labels.csv")
    no_of_vertices=2277
    #-------------------------------------


    frac=.8
    train_lim=math.floor(frac * len(label_data))
    val_lim=train_lim+50
    test_lim=len(label_data)

    #-----------------------------------------




    true_train_edges
    adj=torch.zeros((no_of_vertices+1,no_of_vertices+1))
    adj.shape
    train_edge=torch.tensor(true_train_edges.values)
    for i in range (len(train_edge)):
        adj[train_edge[i][0]][train_edge[i][1]]=1
        adj[train_edge[i][1]][train_edge[i][0]]=1
    adj=sp.coo_matrix(adj)
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    adj = normalize(adj + sp.eye(adj.shape[0]))
    adj = sparse_mx_to_torch_sparse_tensor(adj)
    print('label data--------------------------')
    print(label_data)
    idx=label_data.values[:,0]
    labels=label_data.values[:,1]
    print('here idx------------------------------------')
    print(idx)
    idx_train = idx[:train_lim]
    idx_val = idx[train_lim: val_lim]
    idx_test = idx[val_lim :test_lim]
    
    labels=torch.tensor(labels)
    idx_train=torch.tensor(idx_train)
    idx_val=torch.tensor(idx_val)
    idx_test=torch.tensor(idx_test)
    
    
    features = sp.csr_matrix(np.eye(no_of_vertices+1, dtype=np.float32))
    features = normalize(features)
    features = torch.FloatTensor(np.array(features.todense()))
    
   
    return adj, features, labels, idx_train, idx_val, idx_test
load_my_data()

label data--------------------------
         0  0.1
0        2    1
1        3    1
2        4    2
3        5    2
4        6    3
...    ...  ...
2205  2272    4
2206  2273    4
2207  2274    4
2208  2275    5
2209  2276    0

[2210 rows x 2 columns]
here idx------------------------------------
[   2    3    4 ... 2274 2275 2276]


(tensor(indices=tensor([[   0,  219,  372,  ..., 1982, 2276, 2277],
                        [   0,    0,    0,  ..., 2276, 2276, 2277]]),
        values=tensor([0.2500, 0.1000, 0.2000,  ..., 0.5000, 0.3333, 1.0000]),
        size=(2278, 2278), nnz=9984, layout=torch.sparse_coo),
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 0.,  ..., 0., 0., 0.],
         [0., 0., 1.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 1., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.]]),
 tensor([1, 1, 2,  ..., 4, 5, 0]),
 tensor([   2,    3,    4,  ..., 1819, 1820, 1821]),
 tensor([1822, 1823, 1824, 1825, 1826, 1827, 1828, 1829, 1830, 1831, 1832, 1833,
         1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844, 1845,
         1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855, 1857, 1858,
         1859, 1860, 1861, 1862, 1863, 1864, 1866, 1867, 1868, 1869, 1870, 1871,
         1872, 1873]),
 tensor([1874, 1875, 18

In [0]:
class GraphConvolution(Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'




In [0]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [0]:






# Load data
adj, features, labels, idx_train, idx_val, idx_test = load_my_data()

print("adj-----------------------------------------\n")
print(adj)
print("features-----------------------------------------\n")
print(features)
print(features.size())
print("labels-----------------------------------------\n")
print(labels)
print(labels.size())
print("idx_train-----------------------------------------\n")
print(idx_train)
print("idx_val-----------------------------------------\n")
print(idx_val)
print("idx_test-----------------------------------------\n")
print(idx_test)



# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=10,
            nclass=labels.max().item() + 1,
            dropout=0.5)
optimizer = optim.Adam(model.parameters(),
                       lr=.001, weight_decay=5e-4)

# if args.cuda:
#     model.cuda()
#     features = features.cuda()
#     adj = adj.cuda()
#     labels = labels.cuda()
#     idx_train = idx_train.cuda()
#     idx_val = idx_val.cuda()
#     idx_test = idx_test.cuda()


def train(epoch):
    
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)
    print("output shape"+ str(output.size()))
    z_train=torch.tensor([i for i in range(train_lim)])
    loss_train = F.nll_loss(output[idx_train], labels[z_train])
    acc_train = accuracy(output[idx_train], labels[z_train])
    loss_train.backward()
    optimizer.step()

  
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
    model.eval()
    output = model(features, adj)
    z_val=torch.tensor([i for i in range(train_lim,val_lim)])
#     print(z_val.size())
#     print(output[idx_val].size())
    loss_val = F.nll_loss(output[idx_val], labels[z_val])
    acc_val = accuracy(output[idx_val], labels[z_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    z_test=torch.tensor([i for i in range(val_lim,test_lim)])
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], labels[z_test])
    acc_test = accuracy(output[idx_test], labels[z_test])
    lab=[]
    for i in range(len(idx_test)):
      lab.append(output[idx_test][i].argmax().detach().numpy())
    f1_micro=f1_score(lab, labels[z_test].detach().numpy(),average='micro')
    f1_macro=f1_score(lab, labels[z_test].detach().numpy(),average='macro')
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()),
          "f1_micro= {:.4f}".format(f1_micro),
          "f1_macro= {:.4f}".format(f1_macro),
          
          
          
          )


# Train model
t_total = time.time()
for epoch in range(2000):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))
# Testing
test()

label data--------------------------
         0  0.1
0        2    1
1        3    1
2        4    2
3        5    2
4        6    3
...    ...  ...
2205  2272    4
2206  2273    4
2207  2274    4
2208  2275    5
2209  2276    0

[2210 rows x 2 columns]
here idx------------------------------------
[   2    3    4 ... 2274 2275 2276]
adj-----------------------------------------

tensor(indices=tensor([[   0,  219,  372,  ..., 1982, 2276, 2277],
                       [   0,    0,    0,  ..., 2276, 2276, 2277]]),
       values=tensor([0.2500, 0.1000, 0.2000,  ..., 0.5000, 0.3333, 1.0000]),
       size=(2278, 2278), nnz=9984, layout=torch.sparse_coo)
features-----------------------------------------

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 1., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.]])
torch.Size([2278, 2278])
labels--------

In [0]:
test_edge=true_test_edges.values
res=[]
count=0
output=model(features,adj)
for i in range(len(test_edge)):

    rep1=output[test_edge[i][0]].argmax().detach().numpy()
    rep2=output[test_edge[i][1]].argmax().detach().numpy()

  
    if rep1==rep2 :
      res.append(1)
      
      count+=1
    else:
      res.append(-1)
count/len(test_edge)

0.8339731285988484

In [0]:
test_edge=false_test_edges.values

count=0
output=model(features,adj)
for i in range(len(test_edge)):

    rep1=output[test_edge[i][0]].argmax().detach().numpy()
    rep2=output[test_edge[i][1]].argmax().detach().numpy()

  
    if rep1==rep2 :
      res.append(1)
      
      
    else:
      count+=1
      res.append(-1)
count/len(test_edge)

0.8224568138195777

In [0]:
from sklearn.metrics import f1_score
tr=[1 for i in range(len(true_test_edges))]
for i in range(len(false_test_edges)):
  tr.append(-1)
print(f1_score(tr,res))

0.8291984732824427


In [0]:
no_of_vertices=2277
adj=torch.zeros((no_of_vertices+1,no_of_vertices+1))
adj.shape

train_edge=torch.tensor(true_train_edges.values)
for i in range (len(train_edge)):
    adj[train_edge[i][0]][train_edge[i][1]]=1
    adj[train_edge[i][1]][train_edge[i][0]]=1

In [0]:
wrong_v=np.random.randint(0,no_of_vertices+1,len(train_edge))

In [21]:
train_edge_append=train_edge.clone()
for i in range(len(train_edge)):
    if adj[train_edge[i][0]][wrong_v[i]]==0:
        t=torch.tensor([[train_edge[i][0],wrong_v[i]]])
        train_edge_append=torch.cat((train_edge_append,t))
train_label=[]
for i in range(len(train_edge)):
    train_label.append(1)
for i in range(len(train_edge),len(train_edge_append)):
    train_label.append(-1)
train_label=torch.tensor(train_label)
train_label.size()

torch.Size([8329])